# 4. Inference

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).joinpath('..', 'src').resolve()))

In [3]:
import nasa_space_apps_challenge_2024_seismic_detection as seismic

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `PYTORCH_PRETRAINED_BERT_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `PYTORCH_TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
import pandas as pd
from pprint import pprint
import random

## Parameters

In [5]:
input_row_path = '../data/raw/remote'
input_processed_path = '../data/interim/remote/dataset'
input_models_path = '../models/remote/'

output_path = '../data/interim/remote'

random_state = 42

## 1. Load Model

In [6]:
model = seismic.load_checkpoint(os.path.join(input_models_path, 'moment-fine-tuned-classification-layer'))

/home/eugene/notebooks/competition/nasa_space_apps_challenge_2024_seismic_detection/src/nasa_space_apps_challenge_2024_seismic_detection/models/moment_trainer.py:256: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues 

## Show Record
1. show record
2. predict

### TKTK: Inference

In [8]:
%%time
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

segment_size = 16 * 1024
batch_size = 16  # Adjust batch size based on your system's capability

# Prepare the file list and dataset
file_list = seismic.list_records(input_processed_path)

seismic_dataset = seismic.dataset.SeismicDataset(file_list, input_processed_path, segment_size)
data_loader = DataLoader(seismic_dataset, batch_size=batch_size, shuffle=False)

# # Iterate through the DataLoader and pass batches to the model
for batch_x, batch_labels in data_loader:
    batch_x = batch_x.to(device)
    output = model(x_enc=batch_x)
    predicted_label_idx = output.logits.argmax(dim=1).cpu()

    # Process each prediction in the batch
    for i in range(len(batch_labels)):
        predicted_label = seismic.category_id_to_name[predicted_label_idx[i].item()]
        original_label = batch_labels[i]
        # print(f'Predicted: {predicted_label}')
        # print(f'Original: {seismic.category_id_to_name[original_label.item()]}')
        print(f'{seismic.category_id_to_name[original_label.item()]} / {predicted_label}')
        probabilities = torch.sigmoid(output.logits[i]).cpu().tolist()
        print(f'Probabilities: {probabilities}')
        print()

signal / noise
Probabilities: [0.5241447687149048, 0.48015689849853516]

noise / noise
Probabilities: [0.5241448283195496, 0.4801572263240814]

noise / noise
Probabilities: [0.5241448283195496, 0.4801572859287262]

noise / noise
Probabilities: [0.5241448283195496, 0.4801572263240814]

signal / noise
Probabilities: [0.5241445899009705, 0.48015734553337097]

noise / noise
Probabilities: [0.5241447687149048, 0.48015710711479187]

signal / noise
Probabilities: [0.5241446495056152, 0.4801572263240814]

noise / noise
Probabilities: [0.5241447687149048, 0.4801570177078247]

noise / noise
Probabilities: [0.5241448283195496, 0.4801572263240814]

noise / noise
Probabilities: [0.5241447687149048, 0.48015710711479187]

signal / noise
Probabilities: [0.5241447687149048, 0.48015743494033813]

signal / noise
Probabilities: [0.5241449475288391, 0.4801570177078247]

signal / noise
Probabilities: [0.5241447687149048, 0.48015740513801575]

noise / noise
Probabilities: [0.5241448879241943, 0.4801572263240